This kernel is forked by Andy Harless's kernel "Simple FFNN from Dromosys Features".  
Original kernel uses MinMaxScaler to normalize data.  
I use RankGauss to normalize.  
What is RankGauss?  
RankGauss was used by 1st place solution of "Porto Seguro’s Safe Driver Prediction".   
To know RankGauss in detail, read this https://www.kaggle.com/c/porto-seguro-safe-driver-prediction/discussion/44629 and read this kernel https://www.kaggle.com/rspadim/simple-denoise-autoencoder-with-keras.

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout, BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.optimizers import Adam
from sklearn.model_selection import KFold

import gc
import os
print(os.listdir("../input/"))
print(os.listdir("../input/save-dromosys-features"))
# from keras import backend as KBk
# print(KBk.tensorflow_backend._get_available_gpus())

Using TensorFlow backend.
['previous_application.csv', 'home-credit-default-risk.zip', 'bureau.csv', 'installments_payments.csv', 'credit_card_balance.csv', 'bureau_balance.csv', 'save-dromosys-features', 'HomeCredit_columns_description.csv', 'POS_CASH_balance.csv', 'application_test.csv', 'sample_submission.csv', 'application_train.csv']
['df.pkl.gz']


In [2]:
df = pd.read_pickle('../input/save-dromosys-features/df.pkl.gz')
print("Raw shape: ", df.shape)

y = df['TARGET']
feats = [f for f in df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]
X = df[feats]
print("X shape: ", X.shape, "    y shape:", y.shape)

print("\nPreparing data...")
X = X.fillna(X.mean()).clip(-1e11,1e11)

Raw shape:  (356251, 531)
X shape:  (356251, 528)     y shape: (356251,)

Preparing data...


In [3]:
# i must congrats someone that did this, but i read it on internet, please if it's you, congrats, and explain your code :)
def rank_gauss(x):
    from scipy.special import erfinv
    N = x.shape[0]
    temp = x.argsort()
    rank_x = temp.argsort() / N
    rank_x -= rank_x.mean()
    rank_x *= 2
    efi_x = erfinv(rank_x)
    efi_x -= efi_x.mean()
    return efi_x

In [4]:
for i in X.columns:
    #print('Categorical: ',i)
    X[i] = rank_gauss(X[i].values)

In [5]:
training = y.notnull()
testing = y.isnull()
X_train = X[training].values
X_test = X[testing].values
y_train = np.array(y[training])
print( X_train.shape, X_test.shape, y_train.shape )
gc.collect()

(307507, 528) (48744, 528) (307507,)


22

In [6]:
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback
class roc_callback(Callback):
    def __init__(self,training_data,validation_data):
        self.x = training_data[0]
        self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]


    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(self.x)
        roc = roc_auc_score(self.y, y_pred)
        y_pred_val = self.model.predict(self.x_val)
        roc_val = roc_auc_score(self.y_val, y_pred_val)
        print('\rroc-auc: %s - roc-auc_val: %s' % (str(round(roc,4)),str(round(roc_val,4))),end=100*' '+'\n')
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

In [7]:
folds = KFold(n_splits=10, shuffle=True, random_state=42)
sub_preds = np.zeros(X_test.shape[0])

for n_fold, (trn_idx, val_idx) in enumerate(folds.split(X_train)):
    trn_x, trn_y = X_train[trn_idx], y_train[trn_idx]
    val_x, val_y = X_train[val_idx], y_train[val_idx]

    print('==== KFold:', n_fold, '====')
    print( 'Setting up neural network...' )
    nn = Sequential()
    nn.add(Dense(units = 400 , kernel_initializer = 'normal', input_dim = 528))
    nn.add(PReLU())
    nn.add(Dropout(.3))
    nn.add(Dense(units = 160 , kernel_initializer = 'normal'))
    nn.add(PReLU())
    nn.add(BatchNormalization())
    nn.add(Dropout(.3))
    nn.add(Dense(units = 64 , kernel_initializer = 'normal'))
    nn.add(PReLU())
    nn.add(BatchNormalization())
    nn.add(Dropout(.3))
    nn.add(Dense(units = 26, kernel_initializer = 'normal'))
    nn.add(PReLU())
    nn.add(BatchNormalization())
    nn.add(Dropout(.3))
    nn.add(Dense(units = 12, kernel_initializer = 'normal'))
    nn.add(PReLU())
    nn.add(BatchNormalization())
    nn.add(Dropout(.3))
    nn.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    nn.compile(loss='binary_crossentropy', optimizer='adam')
    
    print( 'Fitting neural network...' )
    nn.fit(trn_x, trn_y, validation_data = (val_x, val_y), epochs=10, verbose=2,
          callbacks=[roc_callback(training_data=(trn_x, trn_y),validation_data=(val_x, val_y))])

    print( 'Predicting...' )
    sub_preds += nn.predict(X_test).flatten().clip(0,1) / folds.n_splits
    
    gc.collect()

0.7767 - roc-auc_val: 0.7743                                                                                                    
Epoch 3/10
 - 93s - loss: 0.2467 - val_loss: 0.2446
roc-auc: 0.7823 - roc-auc_val: 0.7772                                                                                                    
Epoch 4/10
 - 92s - loss: 0.2444 - val_loss: 0.2443
roc-auc: 0.7903 - roc-auc_val: 0.7795                                                                                                    
Epoch 5/10
 - 93s - loss: 0.2430 - val_loss: 0.2435
roc-auc: 0.7946 - roc-auc_val: 0.7783                                                                                                    
Epoch 6/10
 - 91s - loss: 0.2413 - val_loss: 0.2423
roc-auc: 0.8042 - roc-auc_val: 0.7814                                                                                                    
Epoch 7/10
 - 91s - loss: 0.2393 - val_loss: 0.2442
roc-auc: 0.8097 - roc-auc_val: 0.7799                      

In [8]:
print( 'Saving results...' )
sub = pd.DataFrame()
sub['SK_ID_CURR'] = df[testing]['SK_ID_CURR']
sub['TARGET'] = sub_preds
sub[['SK_ID_CURR', 'TARGET']].to_csv('sub_nn.csv', index=False)

print( sub.head() )

Saving results...
        SK_ID_CURR    TARGET
307511      100001  0.026442
307512      100005  0.104763
307513      100013  0.024299
307514      100028  0.055683
307515      100038  0.080555
